# Deep Learning - MCH1
FS23, Manuel Schwarz

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import wandb
from tqdm import tqdm 
from datetime import datetime

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

# sound
import winsound
import time
import datetime

# Schritt 1: Auswahl Task / Datensatz

<div class="alert alert-block alert-info">
    
1. Mache Dir Gedanken, mit welchen Daten Du arbeiten möchtest und welcher Task gelernt werden soll.
    
2. Diskutiere die Idee mit dem Fachcoach.
    
</div>


### Daten und Task
Pytorch stellt einige Datensets zur Verfügung [datasets torch](https://pytorch.org/vision/main/datasets.html).
Verschiedene Kategorien stehen zur Auswahl:
- Image classification
- Image detection or segmentation
- Optical Flow
- Stereo Matching
- Image pairs
- Image captioning
- video classification
- Base classes for custom datasets


**Datenset**  
Eine beliebtes Dateset ist CIFAR10. Es beinhaltet Bilder von 10 Klassen (Flugzeuge, Katzen, Vögel, etc.), die Bilder kommen mit einer Auflösung von 32x32x3 pixel (rgb). Viele Tutorials starten mit diesem Datenset, das lässt darauf schliessen, dass der Rechenaufwand für die Hardware in einem vernümpftigen Rahmen liegt. Daher wird CIFAR10 als Datenset für die Challenge gewählt.

**Task**  
Anhand von CIFAR10 soll ein Modell erstellt werden, welches die Klasse eines Bildes korrekt klassifiziert.



![](cifar10.png)

# Schritt 2: Daten Kennenlernen

<div class="alert alert-block alert-info">
    
1. Mache Dich mit dem Datensatz vertraut, indem Du eine explorative Analyse der Features durchführst: z.B. Vergleich der Klassen pro Feature, Balanciertheit der Klassen. 
2. Führe ein geeignetes Preprocessing durch, z.B. Normalisierung der Daten.  
    
</div>

### 1. Explorative Datenanalyse
[Tutorial Dataloader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [ ]:
# Laden der Daten von CIFAR10
data_path = './data/'
train_data = torchvision.datasets.CIFAR10(data_path, train=True, download=True)

test_data = torchvision.datasets.CIFAR10(data_path, train=False, download=True)

Datengrösse der Trainings- und Testdaten

In [ ]:
print(f'Anzahl Trainingsdaten: {len(train_data)}\n'
      f'Anzahl Testdaten: {len(test_data)}')

**Wie sind die Bilder im Datensatz gespeichert?**  
Die Bilder sind direkt auf dem Datenset via dem Index abrufbar. Ein Tupel mit dem Bild (RGB, 32x32 pixel) und dem Label (6).

In [ ]:
img_0, label_0 = train_data[0]
img_0, label_0

In [ ]:
# deffinieren der Labels CIFAR10
labels_cifar10_dict = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}
labels_cifar10_list = list(labels_cifar10_dict.values())

**Visualisierung der Bilder**  
Die Bilder können mit matplotlib und imshow() direkt vom Dateset über den index dargestellt werden.


In [ ]:
figure = plt.figure(figsize=(6,6))
cols, rows = 5, 5

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_cifar10_list[label], fontsize=8)
    plt.axis('off')
    plt.imshow(img)
plt.show()

**Untersuchen der Verteilungen der Klassen**  
Folgend werden die Verteilungen der Klassen der Cifar10 Datensets auf den Trainings- und Testdaten geprüft.

In [ ]:
train_target = train_data.targets
test_target = test_data.targets
print(f'Labels in Trainingsdaten: {len(train_target)}')
print(f'Labels in Testdaten: {len(test_target)}')

In [ ]:
figure, ax = plt.subplots(1,2, figsize=(10, 3))

ax[0].hist(train_target)
ax[0].set_xticks(np.arange(10))
ax[0].set_xticklabels(labels_cifar10_list, rotation=45)
ax[0].set_title('Trainingset', fontsize=8)

ax[1].hist(test_target)
ax[1].set_xticks(np.arange(10))
ax[1].set_xticklabels(labels_cifar10_list, rotation=45)
ax[1].set_title('Testset', fontsize=8)

plt.suptitle('Verteilung der Klassen von Cifar-10', fontsize=12)
plt.show()

<div class="alert alert-block alert-success">
    
Das Trainingsset umfasst total 50'000 Bilder, davon sind jeweils 5000 Bilder jeder Klasse enthalten. Somit kann zum Beispiel eine Metrik wie'Accuracy' verwendet werden, um die Klassifikation der Modelle zu beurteilen und zu vergleichen. Die 10'000 Bilder in den Testdaten sind ebenfalls gleich verteilt.
    
</div>

### 2. Preprocessing der Daten  
Folgend werden die Daten in einem Preprocessing Schritt für die Modelle vorbereitet:

In [ ]:
# function preprocessing
def preprocessing_cifar10(path='./data/', batch_size=32, 
                          norm_mean=(0.5, 0.5, 0.5), norm_std=(0.5, 0.5, 0.5), 
                          download=True, print_info=False):
    '''
    '''
    if print_info: print(f'------------'), print(f'Preprocessing start')
    # transform tensor to normalized range [-1, 1]
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])
    if print_info: print('Normalized Tensor'), print(f'mean: {norm_mean} | std: {norm_std}')

    
    # CIFAR10: 50000 32x32 color images in 10 classes, with 5000 images per class
    train_dataset = torchvision.datasets.CIFAR10(root=path, train=True,
                                            download=download, transform=transform)

    test_dataset = torchvision.datasets.CIFAR10(root=path, train=False,
                                        download=download, transform=transform)
    if print_info: print('Data transformed')

    # dataloader
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    if print_info: print(f'Dataloader created with {batch_size=}')

    if print_info: print(f'Preprocessing done'), print()   
    return train_dataset, test_dataset, train_loader, test_loader

In [ ]:
# Hyper-parameters 
data_path = './data/'
batch_size = 32

train_dataset, test_dataset, train_loader, test_loader = preprocessing_cifar10(path=data_path, 
                                                                               batch_size=batch_size,
                                                                               norm_mean=(0.5, 0.5, 0.5), 
                                                                               norm_std=(0.5, 0.5, 0.5),
                                                                               download=False,  
                                                                               print_info=True)

# Schritt 3: Aufbau Modellierung  

<div class="alert alert-block alert-info">

1. Lege fest, wie (mit welchen Metriken) Du die Modelle evaluieren möchtest. Berücksichtige auch den Fehler in der Schätzung dieser Metriken.
    
</div>

<div class="alert alert-block alert-success">

Die Klassengrössen sind ausbalanciert, daher ist für die Metrik **Accuracy** für die Modell Beurteilung geieignet und soll hier zum Einsatz kommen:

$$ Accuracy = \frac{Anzahl\ korrekte\ Klassifizierungen}{Total\ Klassifizierungen}  $$

Für eine Klassifizierung von mehr als zwei Klassen eignet sich **CrossEntropyLoss**. Mehr dazu [hier](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html):

$$ \text{CrossEntropyLoss} = -\sum_{i=1}^{N} \left[ y_i \log(\hat{y_i}) + (1 - y_i) \log(1 - \hat{y_i}) \right] $$


Die Accuracy die berechnet wird entspricht einer Schätzung. Bei der Initialisierung der Modellgewichte werden Zufallswerte verwendet. Auch die Batchsize wird durch `shuffle=True` mit unterschiedlicher Aufteilungen erstellt *(siehe `Preprocessing`)*. Somit varriert die Accuracy nach jedem Modeltraining ein wenig. Cross-Validation würde sich hier anbieten, um auf k-folds unterschiedliche Modelle zu erstellen. Mit berechneten Mittelwert $\mu$ und Standardabweichung $\sigma$ kann ein Fehlerabschätzung gemacht werde.  
$$ Fehler_{range} = [\mu - \sigma; \mu + \sigma]$$

</div>

<div class="alert alert-block alert-info">

2. Implementiere Basisfunktionalität, um Modelle zu trainieren und gegeneinander zu evaluieren. Wie sollen die Gewichte initialisiert werden?
    
</div>

<div class="alert alert-block alert-success">

Die Methode wie die Intitialisierung der Gewichte stattfindet hat Einfluss wie schnell die das Modell konvergiert und hilft die Probleme von `vanishing` oder `exploding` Gradienten abzuschwächen. Kleine zufällige Werte führen zu einem effizienteren Trainig. Grosse Werte führen zu Problemenn bei dem das Modell nicht oder nur sehr langsam konvergiert. Je nach Problemstellung und Modelarchitekture können unterschiedliche Initialisierungsmethoden verwendet werden

Mit der Verwendung von `nn.init` (Pytorch) stehen zum Beispiel folgende Optionen zur Verfügung:
1. Uniform initialization
1. Xavier initialization
1. Kaiming initialization
1. Zeros initialization
1. One’s initialization
1. Normal initialization


Die genaue Beschreibung der Optionen kann [hier](https://www.geeksforgeeks.org/initialize-weights-in-pytorch/) gefunden werden. Eine eigene Option stellt Pytorch auch zur Verfügung.

</div>

### Helper Functions  
Um die Erkentnisse dieses Notebook reproduzierbar zu machen, ist es wichtig einen Seed zu definieren. Folgend eine Seed Funktion mit einem üblichen Standart [Quelle](https://vandurajan91.medium.com/random-seeds-and-reproducible-results-in-pytorch-211620301eba).

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

def plot_loss_epoch(num_epochs, loss, figsize=(8, 4)):
    figure = plt.figure(figsize=figsize)
    plt.plot(np.arange(num_epochs), loss)
    plt.xticks(np.arange(num_epochs))
    plt.title('Loss Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

def calc_true_predictions(output_model, true_labels):
    _, predicted = torch.max(output_model.data, 1)
    return (predicted == true_labels).sum().item()

def measure_model_time(start_time, calc='min'):
    model_time = np.round(((time.time() - start_time) / 60), 2)  # from seconds to minute
    return model_time

def play_sound(typ=0):
    # play 'finish' sound
    if typ==0:
        winsound.PlaySound('../01_Dokumentation/win_sounds/beep.wav', winsound.SND_ASYNC)
    if typ==1:
        winsound.PlaySound('../01_Dokumentation/win_sounds/beep2.wav', winsound.SND_ASYNC)

def plot_init_weights(model, figsize=(6, 3)):
    l1_weights = model.state_dict()['linear1.weight'].numpy()
    l2_weights = model.state_dict()['linear2.weight'].numpy()
    fig, ax = plt.subplots(1,2, figsize=figsize)

    ax[0].hist(l1_weights.flatten(), bins=50)
    ax[1].hist(l2_weights.flatten(), bins=50)

    ax[0].set_title("Layer 1", fontsize=8)
    ax[0].set_xlabel("Gewichtswert", fontsize=6)
    ax[0].set_ylabel("Anzahl", fontsize=8)
    ax[0].tick_params(axis='y', labelsize=6)
    ax[0].tick_params(axis='x', labelsize=6)
    ax[1].set_title("Layer 2", fontsize=8)
    ax[1].set_xlabel("Gewichtswert", fontsize=6)
    ax[1].tick_params(axis='y', labelsize=6)
    ax[1].tick_params(axis='x', labelsize=6)
    plt.suptitle(f'Verteilung Initialisierungs Methode {model.init_methode}')
    plt.show()

def test_eval_plot():
    num_epochs = 10
    n_loss_epochs = (0.05 * np.sqrt(np.arange(10))) * -1
    n_correct_train = 0.1 * n_loss_epochs**4
    n_correct_test = 0.05 * n_loss_epochs**4
    # create dataloader
    train_dataset, test_dataset, train_loader, test_loader = preprocessing_cifar10(batch_size=4,
                                                                                norm_mean=(0.5, 0.5, 0.5), 
                                                                                norm_std=(0.5, 0.5, 0.5),
                                                                                download=False,  
                                                                                print_info=False)
    eval_model(num_epochs, n_loss_epochs, n_correct_train, n_correct_test,
                train_loader, test_loader)
# test_eval_plot()   

### Definitionen Evaluation

In [ ]:
def eval_model(num_epochs, n_loss_epochs, n_correct_train, n_correct_test,
               train_loader, test_loader, n_loss_batches=None, id_name='None', 
               figsize=(8,5), print_info=True):
    
    epoch = np.arange(num_epochs)
    acc_train_iter = np.array(n_correct_train) / len(train_loader.dataset)
    acc_test_iter = np.array(n_correct_test) / len(test_loader.dataset)

    fig, ax1 = plt.subplots(figsize=figsize)
    ax2 = ax1.twinx()

    ax1.plot(epoch, n_loss_epochs, color= 'grey', label='Loss')
    ax2.plot(epoch, acc_train_iter, color='steelblue', label='Accuracy Train')
    ax2.plot(epoch, acc_test_iter, color='coral', label='Accuracy Test')
    
    ax1.set_xticks(epoch)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss') 
    ax2.set_ylabel('Accuracy')
    ax1.tick_params(axis='x', labelsize=8)
    ax1.tick_params(axis='y', labelsize=8)
    ax2.tick_params(axis='y', labelsize=8)
    ax1.legend(loc='upper center', bbox_to_anchor=(0.25, -0.12), ncol=3)    
    ax2.legend(loc='upper center', bbox_to_anchor=(0.6, -0.12), ncol=3)
    
    plt.title(f'Training: {id_name}')
    plt.grid()
    plt.show()
    if print_info:
        print(f'Accuracy Train {acc_train_iter[-1]:2f}')
        print(f'Accuracy Test {acc_test_iter[-1]:4f}')
        print(f'Loss Train {n_loss_epochs[-1]:2f}')

    return

### Definitionen Training 

In [ ]:

# Train model function
def train_model(model, train_loader, test_loader,
                loss_func='CrossEntropyLoss', opt='SGD', 
                lr=1e-4, num_epochs=10, id_name='test',
                print_info=False, plot_eval=True, sound=False,
                write_wandb=False):  
    set_seed()

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
    print(f'------------'), print(f'Starten des Trainings auf Device {device}')

    # Model to device
    model = model.to(device)
    model.train()

    # define Loss and Optimizer
    if loss_func == 'CrossEntropyLoss':
        criterion = nn.CrossEntropyLoss()
        if print_info: print(f'Loss Funktion: {loss_func}')
    if opt == 'SGD':        
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        if print_info: print(f'Optimizer: {opt} mit lr: {lr}')

    # train loop
    #n_total_steps = len(train_loader)
    n_loss_epochs = []
    n_loss_all_batches = []
    n_correct_train = []
    n_correct_test = []
    acc_train_epoch = []
    acc_test_epoch = []
    for epoch in tqdm(range(num_epochs)):
        true_train = 0
        n_loss_batch = []
        for i, (images, labels) in enumerate(train_loader):
            # origin shape: [4, 3, 32, 32] = 4, 3, 1024
            # input_layer: 3 input channels, 6 output channels, 5 kernel size
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            n_loss_batch.append(loss.item())
            n_loss_all_batches.append(loss.item())
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            true_train += calc_true_predictions(outputs, labels)
            acc_train = true_train / len(train_loader.dataset)

        # save results
        acc_train = true_train / len(train_loader.dataset)
        epoch_loss = np.mean(n_loss_batch)

        n_correct_train.append(true_train)        
        n_loss_epochs.append(epoch_loss)
        acc_train_epoch.append(acc_train)  

        # eval
        model.eval()
        with torch.no_grad():
            true_val = 0 
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                # Forward pass
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)

                true_val += calc_true_predictions(outputs, labels)                

            n_correct_test.append(true_val)
        model.train()

        acc_test = true_val / len(test_loader.dataset)
        acc_test_epoch.append(acc_test)            
        
        if print_info:
            print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

        # log metrics to wandb
        if write_wandb:
            wandb.log({"loss epoch": epoch_loss, "acc_train": acc_train, "acc_test": acc_test,})

    acc_train = n_correct_train[-1] / len(train_loader.dataset)
    acc_test = n_correct_test[-1] / len(test_loader.dataset)

    if plot_eval:
        eval_model(num_epochs, n_loss_epochs, n_correct_train, n_correct_test,
                   train_loader, test_loader, id_name=id_name, print_info=print_info)
    if sound: play_sound(1)
    print('Finished Training')


    return epoch_loss, acc_train, acc_test

 #if (i+1) % 2000 == 0:
            #    print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

### Einfacher Modell Test  
Folgend soll ein einfaches Modell mit einem hidden Layer erstellt werden um die Basisfunktionen von `train_model()` zu testen. 

Initilisierung Gewichte: 
Da unterschiedliche Problemstellung verschiedene Initialisierungen erfordern, sollen mehrere Methoden ausprobiert werden um die Gewichte zu initialisiern `linear_layer = torch.nn.Linear(2, 3)`:
1. `uniform`: torch.nn.init.uniform_(linear_layer.weight)
1. `xavier`: torch.nn.init.xavier_uniform_(linear_layer.weight)
1. `normal`: torch.nn.init.normal_(linear_layer.weight, mean=0, std=1)
1. `kaiming_norm`: torch.nn.init.kaiming_normal_(linear_layer.weight, nonlinearity="relu")


In [ ]:
class MLPNet_hl1(nn.Module):
    def __init__(self, hidden_l_1:int, act_fn=F.relu, dropout=0, init_methode:str='kaiming_norm'):
        super(MLPNet_hl1, self).__init__()
        self.init_methode = init_methode
        self.linear1 = nn.Linear(3*32*32, hidden_l_1)  # input.shape = (n, 3, 32, 32)
        self.linear1.weight = self.init_dits_weight(self.linear1.weight)
        self.linear2 = nn.Linear(hidden_l_1, 10)
        self.linear2.weight = self.init_dits_weight(self.linear2.weight)
        self.dropout = nn.Dropout(dropout)
        self.activation = act_fn 
       

    def forward(self, x): # x.shape = (n, 3, 32, 32)
        x = x.view(-1, 3*32*32) # x.shape = (n, 3072)
        x = self.activation(self.linear1(x)) # x.shape = (n, 128)
        x = self.dropout(x)
        x = self.linear2(x) # x.shape = (n, 10)

        return x
    
    def init_dits_weight(self, linear_weight):
        if self.init_methode == 'uniform':
            return torch.nn.init.uniform_(linear_weight)
        if self.init_methode == 'xavier':
            return torch.nn.init.xavier_uniform_(linear_weight)
        if self.init_methode == 'normal':
            return torch.nn.init.normal_(linear_weight, mean=0, std=1)
        if self.init_methode == 'kaiming_norm':
            return torch.nn.init.kaiming_normal_(linear_weight, nonlinearity='relu')
   


In [ ]:
method_lst = ['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
# create model
for method in method_lst:
    model = MLPNet_hl1(hidden_l_1=64, init_methode=method)
    plot_init_weights(model, figsize=(5, 1))

Die Verteilungen der Gewichte mit den Methoden `uniform` und  `xavier` sind sehr ähnlich. Mit Xavier ist die X-Skala unterschiedlich, da die Gewichte so skalliert werden dass die Varianz des Output der Varianz des Inputs entspricht.  
Bei `normal` und `kaiming_norm` besteht das gleiche Prinzip, die skallierung der Gewichte, zudem wird die Aktivierungsfunktion berücksichtigt.

In [ ]:
send_data_to_wand = True
exp_name = "simple mlp (1hl)"
project_name = "del-mc1"

#--------- Hyperparameters ---------- 
config = {
    "dataset": "CIFAR-10",
    "name": exp_name,
    "architecture": "MLP", 
    "num_epochs": 20,   
    "batchsize": 64,
    "lr": 1e-3, 
    "opt": 'SGD',
    "loss_func": 'CrossEntropyLoss',
    "hidden_layer_sizes": [128], 
    "kernel_sizes": [],
    "activation": "ReLU",
    "pool_sizes": [],
    "dropout": 0,
    "norm_mean": (0.5, 0.5, 0.5),
    "norm_std": (0.5, 0.5, 0.5),
    "num_classes": 10,
    "init_w_method":"kaiming_norm"  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
}

if send_data_to_wand:
    wandb.init(project=project_name, name=exp_name, notes="", config=config)


# create dataloader
train_dataset, test_dataset, train_loader, test_loader = preprocessing_cifar10(batch_size=config["batchsize"],
                                                                               norm_mean=config["norm_mean"], 
                                                                               norm_std=config["norm_std"],
                                                                               download=False,  
                                                                               print_info=False)
# create model
MLPNet_hl1(hidden_l_1=config["hidden_layer_sizes"][0], 
           init_methode=config["init_w_method"])

# train model
train_model(model, train_loader, test_loader,
            loss_func=config["loss_func"], 
            opt=config["opt"], 
            lr=config["lr"], 
            num_epochs=config["num_epochs"], 
            id_name=config["name"],
            print_info=True, plot_eval=True, sound=True, write_wandb=True)


# Weights & Biases save Parameter
if send_data_to_wand:
    wandb.finish()

Modelltraining Parametereinstellungen mit wandb sweep [Anleitung](https://docs.wandb.ai/ref/python/sweep)

# Schritt 4: Evaluation  

<div class="alert alert-block alert-warning">

Bei der Evaluation ist darauf zu achten, dass das Vorgehen stets möglichst reflektiert erfolgt und versucht wird, die Ergebnisse zu interpretieren. Am Schluss soll auch ein Fazit gezogen werden, darüber welche Variante am besten funktioniert.  

</div>

<div class="alert alert-block alert-info">

**1. Training mit SGD, ohne REG, ohne BN:**  
Untersuche verschiedene Modelle unterschiedlicher Komplexität, welche geeignet sein könnten, um das Klassifikationsproblem zu lösen. Verwende Stochastic Gradient Descent - ohne Beschleunigung, ohne Regularisierung (REG) und ohne Batchnorm (BN).  

a. Für jedes Modell mit gegebener Anzahl Layer und Units pro Layer führe ein sorgfältiges Hyper-Parameter-Tuning durch (Lernrate, Batch-Grösse). Achte stets darauf, dass das Training stabil läuft. Merke Dir bei jedem Training, den Loss, die Performance Metrik(en) inkl. Schätzfehler, die verwendete Anzahl Epochen, Lernrate und Batch-Grösse.

b. Variiere die Anzahl Layer und Anzahl Units pro Layer, um eine möglichst gute Performance zu erreichen. Falls auch CNNs (ohne Transfer-Learning) verwendet werden variiere auch Anzahl Filter, Kernel-Grösse, Stride, Padding.

c. Fasse die Ergebnisse zusammen in einem geeigneten Plot, bilde eine Synthese und folgere, welche Modell-Komplexität Dir am sinnvollsten erscheint.  

</div>


### Deffinition von Modellen


In [ ]:
# Modell Class
class MLPNet_hl1(nn.Module):
    def __init__(self, hidden_l_1:int, act_fn=F.relu, dropout=0, init_methode:str='kaiming_norm'):
        super(MLPNet_hl1, self).__init__()
        self.init_methode = init_methode
        self.linear1 = nn.Linear(3*32*32, hidden_l_1)  # input.shape = (n, 3, 32, 32)
        self.linear1.weight = self.init_dits_weight(self.linear1.weight)
        self.linear2 = nn.Linear(hidden_l_1, 10)
        self.linear2.weight = self.init_dits_weight(self.linear2.weight)
        self.dropout = nn.Dropout(dropout)
        self.activation = act_fn 
       

    def forward(self, x): # x.shape = (n, 3, 32, 32)
        x = x.view(-1, 3*32*32) # x.shape = (n, 3072)
        x = self.activation(self.linear1(x)) # x.shape = (n, 128)
        x = self.dropout(x)
        x = self.linear2(x) # x.shape = (n, 10)

        return x

# Modell Class
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

### Deffinition wandb Trainingloop

In [ ]:
def train_mlp_1hl():
        wandb.init()
        config = wandb.config
        run_name = f'{config["name"]}_lr_{str(config["lr"])}_layer_{str(config["hidden_layer_sizes"])}_bs_{str(config["batchsize"])}'
        print(run_name)
        wandb.run.name = run_name

        # create dataloader
        _, _, train_loader, test_loader = preprocessing_cifar10(batch_size=config["batchsize"],
                                                                                        norm_mean=config["norm_mean"], 
                                                                                        norm_std=config["norm_std"],
                                                                                        download=False,  
                                                                                        print_info=False)
        # create model
        MLPNet_hl1(hidden_l_1=config["hidden_layer_sizes"][0], 
                init_methode=config["init_w_method"])

        # train model
        train_model(model, train_loader, test_loader,
                        loss_func=config["loss_func"], 
                        opt=config["opt"], 
                        lr=config["lr"], 
                        num_epochs=config["num_epochs"], 
                        id_name=config["name"],
                        print_info=False, plot_eval=False, sound=False, write_wandb=True)


### **MLP** Parameter Suche mit wandb

In [ ]:
write_sweep = True
exp_name = "mlp"
num_epochs = 20

sweep_configuration = {
    "name": exp_name,
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "grid",
    "parameters": {"name": {"values": [exp_name]},                    
                    "loss_func": {"values": ['CrossEntropyLoss']},
                    "opt": {"values": ["SGD"]},
                    "init_w_method": {"values": ["kaiming_norm"]},  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
                    "norm_mean": {"values": [(0.5, 0.5, 0.5)]},
                    "norm_std": {"values": [(0.5, 0.5, 0.5)]},
                    "num_epochs": {"values": [num_epochs]},
                    "batchsize": {"values": [64]},
                    "hidden_layer_sizes": {"values": [(32,), (64,)]},
                    "lr": {"values": [1e-3, 1e-4]}
                    },
}

if write_sweep:
    sweep_id = wandb.sweep(sweep_configuration, project="del_mc1_sweeptest")
    wandb.agent(sweep_id, function=train_mlp_1hl)
play_sound(1)

### **CNN** Parameter Suche mit wandb

<div class="alert alert-block alert-info">

**2. Nutzen der Regularisierung**  
Ziehe nun verschiedene Regularisierungsmethoden bei den MLP Layern in Betracht:  
a. L1/L2 Weight Penalty  
b. Dropout

Evaluiere den Nutzen der Regularisierung, auch unter Berücksichtigung verschiedener Regularisierungsstärken. Beschreibe auch kurz, was allgemein das Ziel von Regularisierungsmethoden ist (Regularisierung im Allgemeinen, sowie auch Idee der einzelnen Methoden). Inwiefern wird dieses Ziel im gegebenen Fall erreicht?
    
</div>

<div class="alert alert-block alert-info">

**3. Nutzen von Batchnorm BN (ohne REG, mit SGD)**  
Evaluiere, ob Batchnorm etwas bringt. Beschreibe kurz, was die Idee von BN ist, wozu es helfen soll.
    
</div>    

<div class="alert alert-block alert-info">

**4. Nutzen von Adam (ohne BN, ohne / mit REG)**   
Evaluiere, ob Du mit Adam bessere Resultate erzielen kannst.
    
</div>    

# Tests

<div class="alert alert-block alert-info">
<b>Tipp:</b> Blaue Boxen .. 
</div>

<div class="alert alert-block alert-success">
<b>Erfolg:</b> Grüne Boxen (alert-success)
</div>

<div class="alert alert-block alert-danger">
<b>Warnung:</b> Rote Boxen (alert-danger)
</div>